In [1]:
#train
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import cv2
import random
import time

# 특정 키들
valid_keys = ['w', 'a', 's', 'd']
special_labels = ['no_keys_pressed']

# 데이터셋 클래스 정의
class KeypressDataset(Dataset):
    def __init__(self, base_folders, sequence_length=10, transform=None):
        self.transform = transform
        self.sequence_length = sequence_length
        self.image_sequences = []
        self.label_sequences = []
        self.speed_sequences = []

        for base_folder in base_folders:
            speed_file_path = os.path.join(base_folder, 'text', 'speed.txt')
            with open(speed_file_path, 'r') as f:
                speed_values = [float(line.strip()) for line in f]

            label_folders = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f)) and self.is_valid_label(f)]

            all_image_paths = []
            for label_folder in label_folders:
                folder_path = os.path.join(base_folder, label_folder)
                image_paths = glob.glob(os.path.join(folder_path, '*.png'))
                for img_path in image_paths:
                    all_image_paths.append((img_path, label_folder))

            all_image_paths.sort(key=lambda x: int(os.path.basename(x[0]).split('.')[0]))

            for i in range(len(all_image_paths) - self.sequence_length + 1):
                image_sequence = []
                label_sequence = []
                speed_sequence = []

                for j in range(self.sequence_length):
                    img_path, label_folder = all_image_paths[i + j]
                    label_array = self.label_to_array(label_folder)
                    image_sequence.append(img_path)
                    label_sequence.append(label_array)
                    speed_sequence.append(speed_values[i + j])

                if len(image_sequence) == self.sequence_length:
                    self.image_sequences.append(image_sequence)
                    self.label_sequences.append(label_sequence)
                    self.speed_sequences.append(speed_sequence)

    def __len__(self):
        return len(self.image_sequences)

    def __getitem__(self, idx):
        image_sequence_paths = self.image_sequences[idx]
        label_sequence = self.label_sequences[idx]
        speed_sequence = self.speed_sequences[idx]

        image_sequence = []
        for image_path in image_sequence_paths:
            image = Image.open(image_path).convert('RGB')
            image = np.array(image)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 50, 150)
            image = Image.fromarray(edges).convert('L')
            if self.transform:
                image = self.transform(image)
            image_sequence.append(image)

        image_sequence = torch.stack(image_sequence)
        label_sequence = torch.tensor(label_sequence, dtype=torch.float)
        speed_sequence = torch.tensor(speed_sequence, dtype=torch.float).unsqueeze(1)

        return image_sequence, label_sequence, speed_sequence

    def is_valid_label(self, label):
        if label in special_labels:
            return True
        for char in label.split('_'):
            if char not in valid_keys:
                return False
        return True

    def label_to_array(self, label):
        if label == 'no_keys_pressed':
            return [0, 0, 0, 0]
        array = [0, 0, 0, 0]
        for char in label.split('_'):
            if char in valid_keys:
                index = valid_keys.index(char)
                array[index] = 1
        return array

# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

class CNNLSTMModel(nn.Module):
    def __init__(self, cnn_output_size=256, lstm_hidden_size=128, sequence_length=10):
        super(CNNLSTMModel, self).__init__()
        self.sequence_length = sequence_length
        self.cnn_output_size = cnn_output_size

        # CNN 모델
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, cnn_output_size, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(cnn_output_size)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully Connected 레이어를 추가하여 CNN 출력 크기를 줄임
        self.fc1 = nn.Linear(cnn_output_size * 8 * 8, 512)  # 8*8은 임의의 값, 실제로 확인해야 함
        self.fc2 = nn.Linear(512, 256)

        # LSTM 모델
        self.lstm = nn.LSTM(input_size=256 + 1, hidden_size=lstm_hidden_size, num_layers=2, batch_first=True)
        self.fc3 = nn.Linear(lstm_hidden_size, 4)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, speed_sequence):
        # Check if input x has the expected shape
        if len(x.size()) == 5:
            batch_size, seq_len, c, h, w = x.size()
        else:
            raise ValueError(f"Unexpected input shape: {x.size()}")

        cnn_features = []
        for t in range(seq_len):
            x_t = x[:, t, :, :, :]
            x_t = self.pool(self.relu(self.bn1(self.conv1(x_t))))
            x_t = self.pool(self.relu(self.bn2(self.conv2(x_t))))
            x_t = self.pool(self.relu(self.bn3(self.conv3(x_t))))
            x_t = self.pool(self.relu(self.bn4(self.conv4(x_t))))
            x_t = x_t.view(batch_size, -1)
            x_t = self.relu(self.fc1(x_t))
            x_t = self.relu(self.fc2(x_t))
            cnn_features.append(x_t)

        cnn_features = torch.stack(cnn_features, dim=1)
        lstm_input = torch.cat((cnn_features, speed_sequence), dim=2)


        lstm_out, _ = self.lstm(lstm_input)
        lstm_out = lstm_out[:, -1, :]
        out = self.dropout(lstm_out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out
    
num_epochs = 50

def train_model(base_folders, batch_size=16):
    dataset = KeypressDataset(base_folders=base_folders, transform=transform, sequence_length=10)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    model = CNNLSTMModel().cuda()
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    torch.cuda.empty_cache()
    start_time = time.time()
    for epoch in range(num_epochs):
        running_loss = 0.0
        epoch_start_time = time.time()
        for images, labels, speeds in dataloader:
            images = images.cuda()
            labels = labels.cuda()
            speeds = speeds.cuda()
            optimizer.zero_grad()
            outputs = model(images, speeds)
            loss = criterion(outputs, labels[:, -1, :])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        epoch_end_time = time.time()
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(dataloader)}, Time: {epoch_end_time - epoch_start_time:.2f} seconds')
    total_time = time.time() - start_time
    print(f'Finished Training in {total_time:.2f} seconds')
    torch.save(model.state_dict(), 'cnn_lstm_model.pth')
    print('Model saved to cnn_lstm_model.pth')

def predict_image_sequence(image_sequence, speed_sequence, model):
    image_sequence = torch.stack([transform(Image.fromarray(cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)).convert('L')).unsqueeze(0) for image in image_sequence])
    image_sequence = image_sequence.unsqueeze(0)
    speed_sequence = torch.tensor(speed_sequence, dtype=torch.float).unsqueeze(0).unsqueeze(2)
    with torch.no_grad():
        output = model(image_sequence.cuda(), speed_sequence.cuda())
        output = output.squeeze().cpu().numpy()
    return output

def load_model_and_predict(image_sequence, speed_sequence):
    # image_sequence = torch.stack([transform(Image.fromarray(cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)).convert('L')).unsqueeze(0) for image in image_sequence])
    image_sequence = image_sequence.unsqueeze(0)  # 배치 차원 추가
    speed_sequence = torch.tensor(speed_sequence, dtype=torch.float).unsqueeze(0).unsqueeze(2)  # 배치 차원 및 채널 차원 추가

    model = CNNLSTMModel()
    model.load_state_dict(torch.load('cnn_lstm_model.pth'))
    model.cuda()
    model.eval()

    with torch.no_grad():
        output = model(image_sequence.cuda(), speed_sequence.cuda())
        output = output.squeeze().cpu().numpy()
    return output

if __name__ == '__main__':
    current_directory = os.getcwd()
    base_folders = [os.path.join(current_directory, name) for name in os.listdir(current_directory) if os.path.isdir(os.path.join(current_directory, name))]
    train_model(base_folders, batch_size=16)





example_image_sequence_paths = [
    'screenshots_20240608-050128/no_keys_pressed/1.png',
    'screenshots_20240608-050128/w/2.png',
    'screenshots_20240608-050128/w/3.png',
    'screenshots_20240608-050128/d/4.png',
    'screenshots_20240608-050128/no_keys_pressed/5.png',
    'screenshots_20240608-050128/no_keys_pressed/6.png',
    'screenshots_20240608-050128/no_keys_pressed/7.png',
    'screenshots_20240608-050128/w/8.png',
    'screenshots_20240608-050128/w/9.png',
    'screenshots_20240608-050128/w/10.png'
]
example_speed_sequence = [20, 36, 38, 36, 48, 47, 60, 75, 91, 106]

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_image_sequence = []
for image_path in example_image_sequence_paths:
    image = Image.open(image_path).convert('RGB')
    image = np.array(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    image = Image.fromarray(edges).convert('L')
    image = transform(image)
    test_image_sequence.append(image)

test_image_sequence = torch.stack(test_image_sequence)


# 모델 예측 수행
load_model_and_predict(test_image_sequence, example_speed_sequence)


Epoch 1/50, Loss: 0.38512692917352437, Time: 492.85 seconds
Epoch 2/50, Loss: 0.36971952586338436, Time: 467.49 seconds
Epoch 3/50, Loss: 0.3658609827016962, Time: 463.91 seconds
Epoch 4/50, Loss: 0.36551211346154927, Time: 464.19 seconds
Epoch 5/50, Loss: 0.36311823797637016, Time: 463.85 seconds
Epoch 6/50, Loss: 0.3655607434867442, Time: 464.86 seconds
Epoch 7/50, Loss: 0.3631459799991257, Time: 465.20 seconds
Epoch 8/50, Loss: 0.36492481776352587, Time: 463.71 seconds
Epoch 9/50, Loss: 0.36193369054931335, Time: 465.01 seconds
Epoch 10/50, Loss: 0.3631024405188944, Time: 463.43 seconds
Epoch 11/50, Loss: 0.3643462036190362, Time: 465.11 seconds
Epoch 12/50, Loss: 0.36584447992259056, Time: 463.78 seconds
Epoch 13/50, Loss: 0.3625232120012415, Time: 464.64 seconds
Epoch 14/50, Loss: 0.36582015831580106, Time: 464.46 seconds
Epoch 15/50, Loss: 0.36143635641569377, Time: 465.06 seconds
Epoch 16/50, Loss: 0.365202509466259, Time: 464.46 seconds
Epoch 17/50, Loss: 0.36200880747416925, T

array([0.40370816, 0.10678113, 0.03576658, 0.0974915 ], dtype=float32)

In [8]:
#test
def load_model_and_predict(image_sequence, speed_sequence):
    # image_sequence = torch.stack([transform(Image.fromarray(cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)).convert('L')).unsqueeze(0) for image in image_sequence])
    image_sequence = image_sequence.unsqueeze(0)  # 배치 차원 추가
    speed_sequence = torch.tensor(speed_sequence, dtype=torch.float).unsqueeze(0).unsqueeze(2)  # 배치 차원 및 채널 차원 추가

    model = CNNLSTMModel()
    model.load_state_dict(torch.load('cnn_lstm_model.pth'))
    model.cuda()
    model.eval()

    with torch.no_grad():
        output = model(image_sequence.cuda(), speed_sequence.cuda())
        output = output.squeeze().cpu().numpy()
    return output

    
example_image_sequence_paths = [
    'screenshots_20240608-050128/no_keys_pressed/1.png',
    'screenshots_20240608-050128/w/2.png',
    'screenshots_20240608-050128/w/3.png',
    'screenshots_20240608-050128/d/4.png',
    'screenshots_20240608-050128/no_keys_pressed/5.png',
    'screenshots_20240608-050128/no_keys_pressed/6.png',
    'screenshots_20240608-050128/no_keys_pressed/7.png',
    'screenshots_20240608-050128/w/8.png',
    'screenshots_20240608-050128/w/9.png',
    'screenshots_20240608-050128/w/10.png'
]
example_speed_sequence = [20, 36, 38, 36, 48, 47, 60, 75, 91, 106]

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_image_sequence = []
for image_path in example_image_sequence_paths:
    image = Image.open(image_path).convert('RGB')
    image = np.array(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    image = Image.fromarray(edges).convert('L')
    image = transform(image)
    test_image_sequence.append(image)

test_image_sequence = torch.stack(test_image_sequence)


# 모델 예측 수행
load_model_and_predict(test_image_sequence, example_speed_sequence)

LSTM input shape: torch.Size([1, 10, 257])


array([0.34630337, 0.08237204, 0.02891213, 0.09818259], dtype=float32)

In [3]:
#0611
def load_model_and_predict(image_sequence, speed_sequence):
    # image_sequence = torch.stack([transform(Image.fromarray(cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)).convert('L')).unsqueeze(0) for image in image_sequence])
    image_sequence = image_sequence.unsqueeze(0)  # 배치 차원 추가
    speed_sequence = torch.tensor(speed_sequence, dtype=torch.float).unsqueeze(0).unsqueeze(2)  # 배치 차원 및 채널 차원 추가

    model = CNNLSTMModel()
    model.load_state_dict(torch.load('cnn_lstm_model.pth'))
    model.cuda()
    model.eval()

    with torch.no_grad():
        output = model(image_sequence.cuda(), speed_sequence.cuda())
        output = output.squeeze().cpu().numpy()
    return output

def image_processing(image):
    transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    
    image = np.array(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    image = Image.fromarray(edges).convert('L')
    image = transform(image)

    return image
    
example_image_sequence_paths = [
    'screenshots_20240608-050128/no_keys_pressed/1.png',
    'screenshots_20240608-050128/w/2.png',
    'screenshots_20240608-050128/w/3.png',
    'screenshots_20240608-050128/d/4.png',
    'screenshots_20240608-050128/no_keys_pressed/5.png',
    'screenshots_20240608-050128/no_keys_pressed/6.png',
    'screenshots_20240608-050128/no_keys_pressed/7.png',
    'screenshots_20240608-050128/w/8.png',
    'screenshots_20240608-050128/w/9.png',
    'screenshots_20240608-050128/w/10.png'
]
example_speed_sequence = [20, 36, 38, 36, 48, 47, 60, 75, 91, 106]



test_image_sequence = []
for image_path in example_image_sequence_paths:
    image = Image.open(image_path).convert('RGB')
    image = image_processing(image)

    test_image_sequence.append(image)

test_image_sequence = torch.stack(test_image_sequence)
# print(test_image_sequence)

# 모델 예측 수행
load_model_and_predict(test_image_sequence, example_speed_sequence)

LSTM input shape: torch.Size([1, 10, 257])


array([0.40370816, 0.10678113, 0.03576658, 0.0974915 ], dtype=float32)

In [5]:
# eval
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import cv2
from mss import mss
from pynput import keyboard
from keras.models import load_model
from pynput.keyboard import Key, Controller, Listener

# 특정 키들
valid_keys = ['w', 'a', 's', 'd']
special_labels = ['no_keys_pressed']

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
# ESC 키 종료 플래그
exit_flag = False

def on_press(key):
    global exit_flag
    if key == Key.esc:
        exit_flag = True
        return False

class CNNLSTMModel(nn.Module):
    def __init__(self, cnn_output_size=256, lstm_hidden_size=128, sequence_length=10):
        super(CNNLSTMModel, self).__init__()
        self.sequence_length = sequence_length
        self.cnn_output_size = cnn_output_size

        # CNN 모델
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, cnn_output_size, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(cnn_output_size)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully Connected 레이어를 추가하여 CNN 출력 크기를 줄임
        self.fc1 = nn.Linear(cnn_output_size * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 256)

        # LSTM 모델
        self.lstm = nn.LSTM(input_size=256 + 1, hidden_size=lstm_hidden_size, num_layers=2, batch_first=True)
        self.fc3 = nn.Linear(lstm_hidden_size, 4)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, speed_sequence):
        batch_size, seq_len, c, h, w = x.size()
        cnn_features = []
        for t in range(seq_len):
            x_t = x[:, t, :, :, :]
            x_t = self.pool(self.relu(self.bn1(self.conv1(x_t))))
            x_t = self.pool(self.relu(self.bn2(self.conv2(x_t))))
            x_t = self.pool(self.relu(self.bn3(self.conv3(x_t))))
            x_t = self.pool(self.relu(self.bn4(self.conv4(x_t))))
            x_t = x_t.view(batch_size, -1)
            x_t = self.relu(self.fc1(x_t))
            x_t = self.relu(self.fc2(x_t))
            cnn_features.append(x_t)

        cnn_features = torch.stack(cnn_features, dim=1)
        lstm_input = torch.cat((cnn_features, speed_sequence), dim=2)
        
        # LSTM 입력 크기 확인
        print(f"LSTM input shape: {lstm_input.shape}")

        lstm_out, _ = self.lstm(lstm_input)
        lstm_out = lstm_out[:, -1, :]
        out = self.dropout(lstm_out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

def predict_image_sequence(image_sequence, speed_sequence, model):
    image_sequence = torch.stack([transform(Image.fromarray(cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)).convert('L')).unsqueeze(0) for image in image_sequence])
    speed_sequence = torch.tensor(speed_sequence, dtype=torch.float).unsqueeze(0).unsqueeze(2)
    with torch.no_grad():
        output = model(image_sequence, speed_sequence)
        output = output.squeeze().numpy()
    return output

def load_model_and_predict(image_sequence, speed_sequence):
    image_sequence = torch.stack(image_sequence).unsqueeze(0)  # 배치 차원 추가
    speed_sequence = torch.tensor(speed_sequence, dtype=torch.float).unsqueeze(0).unsqueeze(2)

    model = CNNLSTMModel()
    model.load_state_dict(torch.load('cnn_lstm_model.pth'))
    model.cuda()
    model.eval()

    with torch.no_grad():
        output = model(image_sequence.cuda(), speed_sequence.cuda())
        output = output.squeeze().cpu().numpy()
    return output

def capture_screen(bbox):
    with mss() as sct:
        screenshot = sct.grab(bbox)
        img = Image.frombytes('RGB', (screenshot.width, screenshot.height), screenshot.rgb)
        return img

def get_speed(speed_image, speed_model):
    speed_image = cv2.cvtColor(speed_image, cv2.COLOR_BGR2GRAY)
    speed_image = speed_image.astype('float32') / 255
    speed_image = np.expand_dims(speed_image, axis=0)
    speed_image = np.expand_dims(speed_image, axis=-1)
    prediction = speed_model.predict(speed_image)
    predicted_label = np.argmax(prediction, axis=1)[0]
    return predicted_label

def image_processing(image):
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    image = np.array(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    image = Image.fromarray(edges).convert('L')
    image = transform(image)
    return image

def main():
    global exit_flag
    game_bbox = {'top': 170, 'left': 320, 'width': 1330, 'height': 910}
    speed_bbox = {'top': 905, 'left': 145, 'width': 202 - 145, 'height': 940 - 905}
    
    # 키보드 컨트롤러
    keyboard_controller = Controller()

    speed_model = load_model('speed_rec.h5')
    model = CNNLSTMModel()
    model.load_state_dict(torch.load('cnn_lstm_model.pth'))
    model.cuda()
    model.eval()
    
    image_sequence = []
    speed_sequence = []

    listener = Listener(on_press=on_press)
    listener.start()

    while not exit_flag:
        img = capture_screen(game_bbox)
        image_tensor = image_processing(img)
        image_sequence.append(image_tensor)
        
        speed_screen = capture_screen(speed_bbox)
        speed_screen = np.array(speed_screen)
        speed_screen = cv2.cvtColor(speed_screen, cv2.COLOR_BGR2RGB)
        
        speed = get_speed(speed_screen, speed_model)
        speed_sequence.append(speed)

        if len(image_sequence) == 10:
            prediction = load_model_and_predict(image_sequence, speed_sequence)
            print(f'Predicted keys: {prediction}')
            image_sequence.pop(0)
            speed_sequence.pop(0)

            # 예측 확률에 따른 키 입력
            pressed_keys = []
            for i, prob in enumerate(prediction):
                if prob >= 0.7:
                    key = valid_keys[i]
                    pressed_keys.append(key)
                    keyboard_controller.press(key)
                else:
                    key = valid_keys[i]
                    keyboard_controller.release(key)

            # 콘솔에 속도 값과 입력된 키 표시
            print(f'Speed: {speed}, Keys: {" ".join(pressed_keys)}')

        if exit_flag:
            listener.stop()
            break

if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
LSTM input shape: torch.Size([1, 10, 257])
Predicted keys: [0.40370816 0.10678113 0.03576658 0.0974915 ]
Speed: 23, Keys: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
LSTM input shape: torch.Size([1, 10, 257])
Predicted keys: [0.40370816 0.10678113 0.03576658 0.0974915 ]
Speed: 30, Keys: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
LSTM input shape: torch.Size([1, 10, 257])
Predicted keys: [0.40370816 0.10678113 0.03576658 0.0974915 ]
Speed: 28, Keys: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
LSTM input shape: torch.Size([1, 10, 257])
Predicted keys: [0.40370816 0.10678113 0.03576658 0.0974915 ]
Speed: 26, Keys: 
1/1 ━━━━━━━━━━